In [29]:
import pandas as pd
from pathlib import Path
import importlib
import aspiratio.utils.ir_search as ir_search_module

# Reload to pick up latest changes
importlib.reload(ir_search_module)
from aspiratio.utils.ir_search import search_ir_url

repo = Path('..').resolve()
master_path = repo / 'instrument_master.csv'
coverage_path = repo / 'coverage_table_updated.csv'
master = pd.read_csv(master_path, sep='\t')
coverage = pd.read_csv(coverage_path, sep='\t')

needs_ir = master[master['investor_relations_url'].isna() | (master['investor_relations_url'] == '')]
print(f"Companies needing IR URLs: {len(needs_ir)}")
needs_ir[['CID', 'CompanyName']].head()

Companies needing IR URLs: 355


,CID,CompanyName
36,S37,Acrinova B
37,S38,Actic Group
38,S39,Active Biotech
41,S42,Alimak Group
43,S44,Alligator Bioscience


In [30]:
# Search IR URLs for missing companies
# Score > 200 = high confidence, auto-validate
VALIDATION_THRESHOLD = 200

updated = []
for _, row in needs_ir.iterrows():
    cid = row['CID']
    name = row['CompanyName']
    print(f'Finding IR URL for {name} ({cid})')
    ir_url, score = search_ir_url(name)
    updated.append((cid, ir_url, score))

for cid, ir_url, score in updated:
    if ir_url:
        master.loc[master['CID'] == cid, 'investor_relations_url'] = ir_url
        # Auto-validate if score is above threshold
        if score > VALIDATION_THRESHOLD:
            master.loc[master['CID'] == cid, 'validated'] = True
            print(f"✓ {cid}: Auto-validated (score {score})")

master.to_csv(master_path, sep='\t', index=False)
print(f"\nUpdated {len([u for u in updated if u[1]])} companies with IR URLs")
print(f"Auto-validated {len([u for u in updated if u[2] > VALIDATION_THRESHOLD])} companies (score > {VALIDATION_THRESHOLD})")
master[master['CID'].isin([cid for cid, _, _ in updated])][['CID', 'CompanyName', 'investor_relations_url', 'validated']].head(10)

Finding IR URL for Acrinova B (S37)
Searching for IR URL with queries: ['Acrinova investor relations', 'Acrinova investors', 'Acrinova investor relations site', 'Acrinova financial reports', 'Acrinova investerare', 'Acrinova investors official site', '"Acrinova" investor relations', 'Acrinova årsredovisning']
Searching with DuckDuckGo...
Scored https://www.acrinova.se/hem/investor-relations/: 280 (Preferred TLD boost (.se) (+50), Swedish .se TLD extra boost (+30), Domain segment exact match (+45), IR path/subdomain boost (+40), Company in title (+20), Company in desc (+10), IR keyword in title (+15), IR keyword in desc (+10), Strong IR content validation (+60, 11 terms))
Found high-confidence match (score 280), stopping search.
Best IR URL: https://www.acrinova.se/hem/investor-relations/ (score 280)
Finding IR URL for Actic Group (S38)
Searching for IR URL with queries: ['Actic Group investor relations', 'Actic investors', 'Actic investor relations site', 'Actic financial reports', 'Ac

/var/folders/t2/wr5tn7h97pdb6tkkgvr3xqf00000gn/T/ipykernel_83801/1858165728.py:10: ResourceWarning: unclosed <ssl.SSLSocket fd=103, family=2, type=1, proto=0, laddr=('192.168.0.22', 51883), raddr=('40.114.177.156', 443)>
  ir_url, score = search_ir_url(name)


Scored https://www.acticgroup.se/en/financial-information: 245 (Preferred TLD boost (.se) (+50), Swedish .se TLD extra boost (+30), Group boost (+30), Domain segment substring match (+40), Company in title (+20), IR keyword in title (+15), Strong IR content validation (+60, 11 terms))
Found high-confidence match (score 245), stopping search.
Best IR URL: https://www.acticgroup.se/en/financial-information (score 245)
Finding IR URL for Active Biotech (S39)
Searching for IR URL with queries: ['Active Biotech investor relations', 'Active Biotech investors', 'Active Biotech investor relations site', 'Active Biotech financial reports', 'Active Biotech investerare', 'Active Biotech investors official site', '"Active Biotech" investor relations', 'Active Biotech årsredovisning']
Searching with DuckDuckGo...


/var/folders/t2/wr5tn7h97pdb6tkkgvr3xqf00000gn/T/ipykernel_83801/1858165728.py:10: ResourceWarning: unclosed <ssl.SSLSocket fd=99, family=2, type=1, proto=0, laddr=('192.168.0.22', 51900), raddr=('40.114.177.156', 443)>
  ir_url, score = search_ir_url(name)


Scored http://investors.aduro.com.cutestat.com/: 115 (Preferred TLD boost (.com) (+50), Subdomain boost (investors) (+40), IR keyword in title (+15), Weak IR content validation (+10, 2 terms))
Found high-confidence match (score 115), stopping search.
Best IR URL: http://investors.aduro.com.cutestat.com/ (score 115)
Finding IR URL for Alimak Group (S42)
Searching for IR URL with queries: ['Alimak Group investor relations', 'Alimak investors', 'Alimak investor relations site', 'Alimak financial reports', 'Alimak investerare', 'Alimak Group investors official site', '"Alimak" investor relations', 'Alimak årsredovisning']
Searching with DuckDuckGo...


...

/Users/jakobjohannesson/Documents/github_repos/aspiratio/.venv/lib/python3.13/site-packages/hyperframe/flags.py:25: ResourceWarning: unclosed <ssl.SSLSocket fd=99, family=2, type=1, proto=0, laddr=('192.168.0.22', 52205), raddr=('40.114.177.156', 443)>
  self._valid_flags = {flag.name for flag in defined_flags}
/Users/jakobjohannesson/Documents/github_repos/aspiratio/.venv/lib/python3.13/site-packages/hyperframe/flags.py:25: ResourceWarning: unclosed <ssl.SSLSocket fd=109, family=2, type=1, proto=0, laddr=('192.168.0.22', 52247), raddr=('40.114.177.156', 443)>
  self._valid_flags = {flag.name for flag in defined_flags}


DuckDuckGo search error for query 'Arjo årsredovisning': No results found.
Few results, trying DuckDuckGo with Swedish region...
DuckDuckGo SE search error: No results found.
DuckDuckGo SE search error: No results found.
DuckDuckGo SE search error: No results found.
No valid IR URL found for Arjo B
Finding IR URL for Arla Plast (S54)
Searching for IR URL with queries: ['Arla Plast investor relations', 'Arla Plast investors', 'Arla Plast investor relations site', 'Arla Plast financial reports', 'Arla Plast investerare', 'Arla Plast investors official site', '"Arla Plast" investor relations', 'Arla Plast årsredovisning']
Searching with DuckDuckGo...
DuckDuckGo search error for query 'Arla Plast investor relations': No results found.
DuckDuckGo search error for query 'Arla Plast investors': No results found.
DuckDuckGo search error for query 'Arla Plast investor relations site': No results found.
DuckDuckGo search error for query 'Arla Plast financial reports': No results found.
DuckDuckG

/opt/homebrew/Cellar/python@3.13/3.13.1/Frameworks/Python.framework/Versions/3.13/lib/python3.13/html/parser.py:316: ResourceWarning: unclosed <ssl.SSLSocket fd=110, family=2, type=1, proto=0, laddr=('192.168.0.22', 52270)>
  m = attrfind_tolerant.match(rawdata, k)


Scored https://www.ascelia.com/mfn_news/ascelia-pharma-announces-management-changes-to-support-future-growth/: 120 (Preferred TLD boost (.com) (+50), Long path penalty (-20), Company in title (+20), Company in desc (+10), Strong IR content validation (+60, 7 terms))
Found high-confidence match (score 120), stopping search.
Best IR URL: https://www.ascelia.com/mfn_news/ascelia-pharma-announces-management-changes-to-support-future-growth/ (score 120)
Finding IR URL for Asker Healthcare Group (S56)
Searching for IR URL with queries: ['Asker Healthcare Group investor relations', 'Asker Healthcare investors', 'Asker Healthcare investor relations site', 'Asker Healthcare financial reports', 'Asker Healthcare investerare', 'Asker Healthcare Group investors official site', '"Asker Healthcare" investor relations', 'Asker Healthcare årsredovisning']
Searching with DuckDuckGo...
Scored https://www.asker.com/: 155 (Preferred TLD boost (.com) (+50), Company in title (+20), IR keyword in title (+15)

KeyboardInterrupt: 

In [ ]:
# Sync coverage table IR URLs from master
coverage = coverage.merge(master[['CID', 'investor_relations_url']], left_on='Company_Identifier', right_on='CID', how='left')
coverage['IR_URL'] = coverage['investor_relations_url']
coverage.drop(columns=['CID', 'investor_relations_url'], inplace=True)
coverage.to_csv(coverage_path, sep='\t', index=False)
coverage[['CompanyName', 'FiscalYear', 'IR_URL']].head()

,CompanyName,FiscalYear,IR_URL
0,ABB Ltd,2019,https://global.abb/group/en/investors/annual-r...
1,ABB Ltd,2020,https://global.abb/group/en/investors/annual-r...
2,ABB Ltd,2021,https://global.abb/group/en/investors/annual-r...
3,ABB Ltd,2022,https://global.abb/group/en/investors/annual-r...
4,ABB Ltd,2023,https://global.abb/group/en/investors/annual-r...
